In [7]:
import os
%pwd

'C:\\Users\\SGU\\Projects\\mlops-with-mlflow'

In [8]:
os.chdir(r"C:\Users\SGU\Projects\mlops-with-mlflow")
%pwd

'C:\\Users\\SGU\\Projects\\mlops-with-mlflow'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [10]:
from mlProject.utils.constants import CONFIG_FILE_PATH, SCHEMA_FILE_PATH, PARAMS_FILE_PATH
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self) -> None:
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
        print(self.config)
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        return DataTransformationConfig(
            config.root_dir,
            config.data_path,
        )
        
        

In [11]:

import pandas as pd
from sklearn.model_selection import train_test_split
from mlProject import logger
class DataTransformation:
    def __init__(self, config: DataTransformationConfig) -> None:
        self.config = config
        
    def train_test_split(self):
        data = pd.read_csv(self.config.data_path)
        
        train_df, test_df = train_test_split(data, test_size=0.2)
        train_df.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test_df.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
        
        logger.info("splitted data into train and test set")
        logger.info(train_df.shape)
        logger.info(test_df.shape)
        
        

In [12]:
try:
    cfg_mgr = ConfigurationManager()
    data_transformation_config = cfg_mgr.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.train_test_split()
except Exception as e:
    raise e

[2023-08-12 07:34:59,190: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-12 07:34:59,216: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-12 07:34:59,231: INFO: common: yaml file: params.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_path': 'artifacts/data_ingestion/winequality-red.csv', 'status_file': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/winequality-red.csv'}}
[2023-08-12 07:34:59,237: INFO: common: created directory: artifacts]
[2023-08-12 07:34:59,240: INFO: common: created directory: artifacts\da